# Week VII: Genome Annotation & BLAST

## Open Reading Frame (ORF)

In genomics, an **open reading frame (ORF)** refers to a segment of a DNA sequence lacking a stop codon, which acts as a signal to halt protein synthesis. A codon is a trinucleotide in DNA or RNA, representing a unit of genetic information that encodes either an amino acid or signals the termination of protein synthesis (known as a stop codon). Among the 64 codons, 61 specify amino acids, while 3 serve as stop codons. Frequently, an extended open reading frame constitutes a component of a gene, specifically a sequence directly responsible for encoding a protein.

<img src=https://www.genome.gov/sites/default/files/media/images/tg/Open-reading-frame.jpg width="1200" class="center">

**Figure 1.** Open Reading Frame (ORF) (From [NIH Genetics Glossary](https://www.genome.gov/genetics-glossary/Open-Reading-Frame))

Due to the fact that codons consist of three nucleic acids and DNA is double-stranded, the ribosome has the capability to interpret an RNA derived from either strand. Moreover, it can read the RNA in sequential sets of three (1-2-3), with these sets being distinct and isolated. Consequently, there are three potential reading ORFs in one direction and an additional three in the opposite direction. This results in a total of six distinct reading ORFs for each DNA segment, offering multiple possibilities for the existence of an open reading frame.

<img src="ORFs.jpg" width="1200" class="center">

**Figure 2.** Six reading ORFs of an DNA sequence.

### Identifying open reading ORFs

We’re going to extract the six frame translation. As explanation this is how we extract it:

Consider the sequence in Figure 2:

5'-TCTCTACGATGCTGAAAATTGTTACTCGGGCTGGACACACAGCTAGAATATCGTGAA-3'  
3'-AGAGATGCTACGACTTTTAACAATGAGCCCGACCTGTGTGTCGATCTTATAGCACTT-5'

Divide the sequence into 6 different reading frames (+1, +2, +3, -1, -2 and -3). The first reading frame is obtained by considering the sequence in words of 3.

FRAME +1: **TCT CTA CGA TGC TGA AAA TTG TTA CTC GGG CTG GAC ACA CAG CTA GAA TAT CGT GAA**

The second reading frame is formed after leaving the first nucleotide and then grouping the sequence into words of 3 nucleotides:

FRAME +2: T **CTC TAC GAT GCT GAA AAT TGT TAC TCG GGC TGG ACA CAC AGC TAG AAT ATC GTG** AA

The third reading frame is formed after leaving the first 2 nucleotides and then grouping the sequence into words of 3 nucleotides:

FRAME +3: TC **TCT ACG ATG CTG AAA ATT GTT ACT CGG GCT GGA CAC ACA GCT AGA ATA TCG TGA** A


The other 3 reading frames can be found only after finding the reverse complement.

Reverse complement:  5'-TTCACGATATTCTAGCTGTGTGTCCAGCCCGAGTAACAATTTTCAGCATCGTAGAGA-3'

Now same process as that of +1, +2 and +3 strands is repeated for -1, -2 and -3 strands with reverse complement sequence

FRAME -1: **TTC ACG ATA TTC TAG CTG TGT GTC CAG CCC GAG TAA CAA TTT TCA GCA TCG TAG AGA**

FRAME -2: T **TCA CGA TAT TCT AGC TGT GTG TCC AGC CCG AGT AAC AAT TTT CAG CAT CGT AGA** GA

FRAME -3: TT **CAC GAT ATT CTA GCT GTG TGT CCA GCC CGA GTA ACA ATT TTC AGC ATC GTA GAG** A

In [ ]:
from Bio.Seq import Seq
my_seq = Seq("TCTCTACGATGCTGAAAATTGTTACTCGGGCTGGACACACAGCTAGAATATCGTGAA")
complement = my_seq.complement()
print(complement)
reversecomplement = my_seq.reverse_complement()
print(reversecomplement)

Now lets write a code to find 6 reading frames with python:

In [34]:
from Bio.Seq import Seq

dna="TCTCTACGATGCTGAAAATTGTTACTCGGGCTGGACACACAGCTAGAATATCGTGAA"
listofORF = list()

frames = [] # storing the six frames that it should be extacted from the sequence

# create the positive frames
# split the frames into codons for better performance
frames.append([dna[i:i + 3] for i in range(0, len(dna), 3)])
frames.append([dna[i:i + 3] for i in range(1, len(dna), 3)])
frames.append([dna[i:i + 3] for i in range(2, len(dna), 3)])

# reverse complement of the fragment
rc_dna = my_seq.reverse_complement()

# create the negative frames
frames.append([rc_dna[i:i + 3] for i in range(0, len(rc_dna), 3)])
frames.append([rc_dna[i:i + 3] for i in range(1, len(rc_dna), 3)])
frames.append([rc_dna[i:i + 3] for i in range(2, len(rc_dna), 3)])

The last step consist of another loop, where we’re going to loop the 6 open reading frames and extract ORFs.

In [35]:
for i in range(0,len(frames),1): #looping all the frames
    start=0
    while start <len(frames[i]): #looping each frame for start and stop codons 
        if frames[i][start]=="ATG": #or Frames[i][start]=="TTG" or Frames[i][start]=="CTG" or Frames[i][start]=="GTG":
            for stop in range(start+1,len(frames[i]),1):
                         if frames[i][stop]=="TAA" or  frames[i][stop]=="TAG" or  frames[i][stop]=="TGA" :
                                listofORF.append(frames[i][start:stop]) # retrieve the ORF
                                start=stop+1 # avoiding multiple start codons
                                break
        start+=1

Now all the extracted ORFs are stored in listofORF list.

In [ ]:
print(listofORF)
listofORF_seq = [''.join(ORF) for ORF in listofORF]
print(listofORF_seq)
ORFofinterest = listofORF_seq[0]
print(ORFofinterest)

### Real Example

A very simplistic first step at identifying possible genes is to look for open reading frames (ORFs). By this we mean look in all six frames for long regions without stop codons – an ORF is just a region of nucleotides with no in frame stop codons.

Of course, to find a gene you would also need to worry about locating a start codon, possible promoters – and in Eukaryotes there are introns to worry about too. However, this approach is still useful in viruses and Prokaryotes.

To show how you might approach this with Biopython, we’ll need a sequence to search, and as an example we’ll again use the bacterial plasmid – although this time we’ll start with a plain FASTA file with no pre-marked genes: [NC_005816.fna](https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.fna). This is a bacterial sequence, so we’ll want to use NCBI Codon Table 11.

Here is a neat trick using the Seq object’s split method to get a list of all the possible ORF translations in the six reading frames:

In [ ]:
from Bio import SeqIO
record = SeqIO.read("NC_005816.fna", "fasta")
table = 11
min_protein_length = 100

frames = []
listofORF = []
listofProtein = []
for strand, r_seq in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
    for frame_no in range(3):
        seq = str(r_seq)
        frame = [seq[i:i + 3] for i in range(frame_no, len(seq), 3)]
        frames.append(''.join(frame))
        start=0
        while start <len(frame):
            if frame[start]=="ATG":
                for stop in range(start+1,len(frame),1):
                    if frame[stop]=="TAA" or frame[stop]=="TAG" or  frame[stop]=="TGA":
                        ORF = ''.join(frame[start:stop])
                        listofORF.append(ORF) # retrieve the ORF
                        ORF_seq = Seq(ORF)
                        protein = ORF_seq.translate(table)
                        if len(protein) >= min_protein_length:
                            listofProtein.append(str(protein))
                            print(
                                "%s...%s - length %i, strand %i, frame %i"
                                % (protein[:50], protein[-50:], len(protein), strand, frame_no)
                                )
                        start=stop+1 # avoiding multiple start codons
                        break
            start+=1

## BLAST (Basic Local Alignment Search Tool)

BLAST, which stands for Basic Local Alignment Search Tool, is a program designed to identify regions of similarity between biological sequences. This tool is employed to compare nucleotide or protein sequences against databases, and it assesses the statistical significance of the identified matches.  

By utilizing BLAST, researchers can draw inferences about functional and evolutionary relationships between sequences. Additionally, the program aids in the identification of members within gene families. In essence, BLAST serves as a valuable resource for exploring the similarities and connections present in biological sequences.

Fortunately, the Biopython folks have developed lots of tools for dealing with BLAST and making things much easier. This section details how to use these tools and do useful things with them.

Dealing with BLAST can be split up into two steps, both of which can be done from within Biopython. Firstly, running BLAST for your query sequence(s), and getting some output. Secondly, parsing the BLAST output in Python for further analysis.

Your first introduction to running BLAST was probably via the NCBI web-service. In fact, there are lots of ways you can run BLAST, which can be categorized in several ways. The most important distinction is running BLAST locally (on your own machine), and running BLAST remotely (on another machine, typically the NCBI servers).

### Running BLAST via the NCBI web-service

* Go to the [BLAST web page](https://blast.ncbi.nlm.nih.gov/Blast.cgi)
* Choose Nucleotide BLAST
* Copy and paste a small part of the FASTA formatted sequence in [NC_005816.fna](https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.fna) file into **"Enter Query Sequence"** box.
* Click **"Show results in a new window"** checkbox.
* Click **BLAST** button at the bottom of the page to run BLAST with default parameters.
* Discuss the results in **Descriptions**, **Graphic Summary**, **Alignments** and **Taxonomy** tabs.

### Running BLAST over the Internet

We use the function `qblast()` in the `Bio.Blast.NCBIWWW` module to call the online version of BLAST. This has three non-optional arguments:

* The first argument is the blast program to use for the search, as a lower case string. The options and descriptions of the programs are available at https://blast.ncbi.nlm.nih.gov/Blast.cgi. Currently `qblast` only works with `blastn`, `blastp`, `blastx`, `tblast` and `tblastx`.
* The second argument specifies the databases to search against. Again, the options for this are available on the [NCBI Guide](ftp://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf) to BLAST.
* The third argument is a string containing your query sequence. This can either be the sequence itself, the sequence in fasta format, or an identifier like a GI number.

[NCBI guidelines](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=DeveloperInfo) state:

1. Do not contact the server more often than once every 10 seconds.
2. Do not poll for any single RID more often than once a minute.
3. Use the URL parameter email and tool, so that the NCBI can contact you if there is a problem.
4. Run scripts weekends or between 9 pm and 5 am Eastern time on weekdays if more than 50 searches will be submitted.

To fulfill the third point, one can set the NCBIWWW.email variable.

In [90]:
from Bio.Blast import NCBIWWW
NCBIWWW.email = "namesurname@posta.mu.edu.tr"

If you have a nucleotide sequence you want to search against the nucleotide database (nt) using BLASTN, and you know the GI number of your query sequence, you can use:

In [95]:
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")

Alternatively, if we have our query sequence already in a [FASTA formatted file](https://raw.githubusercontent.com/biopython/biopython/master/Doc/examples/m_cold.fasta), we just need to open the file and read in this record as a string, and use that as the query argument:
```
from Bio.Blast import NCBIWWW
fasta_string = open("m_cold.fasta").read()
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
```

We could also have read in the FASTA file as a SeqRecord and then supplied just the sequence itself:
```
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("m_cold.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)
```

Supplying just the sequence means that BLAST will assign an identifier for your sequence automatically. You might prefer to use the SeqRecord object’s format method to make a FASTA string (which will include the existing identifier):
```
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("m_cold.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))
```

This approach makes more sense if you have your sequence(s) in a non-FASTA file format which you can extract using Bio.SeqIO.

Whatever arguments you give the qblast() function, you should get back your results in a handle object (by default in XML format). The next step would be to parse the XML output into Python objects representing the search results, but you might want to save a local copy of the output file first. I find this especially useful when debugging my code that extracts info from the BLAST results (because re-running the online search is slow and wastes the NCBI computer time).

We need to be a bit careful since we can use result_handle.read() to read the BLAST output only once – calling result_handle.read() again returns an empty string.
```
with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

result_handle.close()
```

After doing this, the results are in the file my_blast.xml and the original handle has had all its data extracted (so we closed it). However, the parse function of the BLAST parser (described in ‍7.3) takes a file-handle-like object, so we can just open the saved file for input:
```
result_handle = open("my_blast.xml")
```

Now that we’ve got the BLAST results back into a handle again, we are ready to do something with them, so this leads us right into the parsing section.

### Running BLAST locally

Running BLAST locally (as opposed to over the internet) has at least major two advantages:

* Local BLAST may be faster than BLAST over the internet;
* Local BLAST allows you to make your own database to search for sequences against.

Dealing with proprietary or unpublished sequence data can be another reason to run BLAST locally. You may not be allowed to redistribute the sequences, so submitting them to the NCBI as a BLAST query would not be an option.

Unfortunately, there are some major drawbacks too – installing all the bits and getting it setup right takes some effort:

* Local BLAST requires command line tools to be installed.
* Local BLAST requires (large) BLAST databases to be setup (and potentially kept up to date).
To further confuse matters there are several different BLAST packages available, and there are also other tools which can produce imitation BLAST output files, such as BLAT.

#### Standalone NCBI BLAST+

The “new” NCBI BLAST+ suite was released in 2009. This replaces the old NCBI “legacy” BLAST package.

This section will show briefly how to use these tools from within Python. First, we construct a command line string (as you would type in at the command line prompt if running standalone BLAST by hand). Then we can execute this command from within Python.

For example, taking a FASTA file of gene nucleotide sequences, you might want to run a BLASTX (translation) search against the non-redundant (NR) protein database. Assuming you (or your systems administrator) has downloaded and installed the NR database, you might run in terminal:
```
blastx -query opuntia.fasta -db nr -out opuntia.xml -evalue 0.001 -outfmt 5
```
This should run BLASTX against the NR database, using an **expectation cut-off value (E-value)** of 0.001 and produce XML output to the specified file (which we can then parse). On my computer this takes about six minutes - a good reason to save the output to a file so you can repeat any analysis as needed.

From within python we can use the subprocess module to build the command line string, and run it:
```
import subprocess
cmd = "blastx -query opuntia.fasta -db nr -out opuntia.xml"
cmd += " -evalue 0.001 -outfmt 5"
subprocess.run(cmd, shell=True)
```
In this example there shouldn’t be any output from BLASTX to the terminal. You may want to check the output file opuntia.xml has been created.

### Parsing BLAST output

As mentioned above, BLAST can generate output in various formats, such as **XML**, **HTML**, and **plain text**. Originally, Biopython had parsers for BLAST plain text and HTML output, as these were the only output formats offered at the time. Unfortunately, the BLAST output in these formats kept changing, each time breaking the Biopython parsers. Our HTML BLAST parser has been removed, while the deprecated plain text BLAST parser is now only available via `Bio.SearchIO`. Use it at your own risk, it may or may not work, depending on which BLAST version you’re using.

As keeping up with changes in BLAST became a hopeless endeavor, especially with users running different BLAST versions, **we now recommend to parse the output in XML format**, which can be generated by recent versions of BLAST. Not only is the XML output more stable than the plain text and HTML output, it is also much easier to parse automatically, making Biopython a whole lot more stable.

You can get BLAST output in XML format in various ways. For the parser, it doesn’t matter how the output was generated, as long as it is in the XML format.

* You can use Biopython to run BLAST over the internet.
* You can use Biopython to run BLAST locally.
* You can do the BLAST search yourself on the NCBI site through your web browser, and then save the results. You need to choose XML as the format in which to receive the results, and save the final BLAST page you get (you know, the one with all of the interesting results!) to a file.
* You can also run BLAST locally without using Biopython, and save the output in a file. Again, you need to choose XML as the format in which to receive the results.

The important point is that you do not have to use Biopython scripts to fetch the data in order to be able to parse it. Doing things in one of these ways, you then need to get a handle to the results. In Python, a handle is just a nice general way of describing input to any info source so that the info can be retrieved using `read()` and `readline()` functions.

If you followed the code above for interacting with BLAST through a script, then you already have result_handle, the handle to the BLAST results. For example, using a GI number to do an online search:

Save results to a file, for parsing:

In [ ]:
import os.path
from Bio.Blast import NCBIWWW
if not os.path.exists("blastn_8332116.xml"):
    result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")
    blast_results = result_handle.read()
    result_handle.close()
    save_file = open("blastn_8332116.xml", "w")
    save_file.write(blast_results)
    save_file.close()
# Do something with the blast results in blastn-nr-8332116.xml

If instead you ran BLAST some other way, and have the BLAST output (in XML format) in the file *my_blast.xml*, all you need to do is to open the file for reading:

In [ ]:
result_handle = open("blastn_8332116.xml")

Now that we’ve got a handle, we are ready to parse the output. The code to parse it is really quite small. If you expect a single BLAST result (i.e., you used a single query):

In [ ]:
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

Or, if you have lots of results (i.e., multiple query sequences):

In [ ]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)

Just like Bio.SeqIO and Bio.AlignIO (see Chapters ‍5 and ‍6), we have a pair of input functions, read and parse, where read is for when you have exactly one object, and parse is an iterator for when you can have lots of objects – but instead of getting SeqRecord or MultipleSeqAlignment objects, we get BLAST record objects.

To be able to handle the situation where the BLAST file may be huge, containing thousands of results, NCBIXML.parse() returns an iterator. In plain English, an iterator allows you to step through the BLAST output, retrieving BLAST records one by one for each BLAST search result:

In [ ]:
for blast_record in blast_records:
    pass  # Do something with blast_record

Note though that you can step through the BLAST records only once. Usually, from each BLAST record you would save the information that you are interested in. If you want to save all returned BLAST records, you can convert the iterator into a list:

In [ ]:
blast_records = list(blast_records)

Now you can access each BLAST record in the list with an index as usual. If your BLAST file is huge though, you may run into memory problems trying to save them all in a list.

Usually, you’ll be running one BLAST search at a time. Then, all you need to do is to pick up the first (and only) BLAST record in blast_records:

In [ ]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)
blast_record = next(blast_records)

Or more elegantly:

In [ ]:
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

I guess by now you’re wondering what is in a BLAST record.

### The BLAST record class

A BLAST Record contains everything you might ever want to extract from the BLAST output. Right now we’ll just show an example of how to get some info out of the BLAST report, but if you want something in particular that is not described here, look at the info on the record class in detail, and take a gander into the code or automatically generated documentation – the docstrings have lots of good info about what is stored in each piece of information.

To continue with our example, let’s just print out some summary info about all hits in our blast report greater than a particular threshold. The following code does this:

In [ ]:
E_value_threshold = 0.04

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_value_threshold:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

In [ ]:
from Bio.Blast import NCBIXML
E_value_threshold = 1e-5
result_handle = open("blastn_8332116.xml")
for blast_result in NCBIXML.parse(result_handle):
    for desc in blast_result.descriptions:
        if desc.e < E_value_threshold:
            print('****Alignment****')
            print('sequence:', desc.title)
            print('e value:', desc.e)

This will print out summary reports like the following:
```
****Alignment****
sequence: >gb|AF283004.1|AF283004 Arabidopsis thaliana cold acclimation protein WCOR413-like protein
alpha form mRNA, complete cds
length: 783
e value: 0.034
tacttgttgatattggatcgaacaaactggagaaccaacatgctcacgtcacttttagtcccttacatattcctc...  
||||||||| | ||||||||||| || ||||  || || |||||||| |||||| |  | |||||||| ||| ||...  
tacttgttggtgttggatcgaaccaattggaagacgaatatgctcacatcacttctcattccttacatcttcttc...  
```
Basically, you can do anything you want to with the info in the BLAST report once you have parsed it. This will, of course, depend on what you want to use it for, but hopefully this helps you get started on doing what you need to do!

An important consideration for extracting information from a BLAST report is the type of objects that the information is stored in. In Biopython, the parsers return Record objects, either Blast or PSIBlast depending on what you are parsing. These objects are defined in `Bio.Blast.Record` and are quite complete.

The Blast class diagram is shown below:

<img src=https://biopython.org/DIST/docs/tutorial/images/BlastRecord.png width="600" class="center">

**Figure 3.** The diagram of BLAST class (From [Biopython Tutorial and Cookbook](https://biopython.org/DIST/docs/tutorial/Tutorial.html))

You can get any info stored in BLAST record using the hierarchy of the record shown in the diagram (Figure 3).

## Homework

Prepare a Jupyter notebook to achive the goals below and upload to 'Homeworks/Week_07' folder under Google Drive directory of the course.
* Search both ORF sequences in DNA and translated Protein of [NC_005816.fna](https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.fna) file using BLAST